In [1]:
%autosave 180

Autosaving every 180 seconds


In [2]:
%matplotlib inline
import numpy as np
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
from astropy.utils.data import get_pkg_data_filename
import galsim
import os
import math
import logging
import sys
from astropy.io import fits
from astropy.visualization import astropy_mpl_style
from astropy.coordinates import SphericalRepresentation, CartesianRepresentation

In [3]:
simname = "/nfs/slac/des/fs1/g/sims/jderose/BCC/Chinchilla/Herd/Chinchilla-4/v1.9.2/addgalspostprocess/mag2/Chinchilla-4-y3wlpz.0.fits"
truthname = "/nfs/slac/des/fs1/g/sims/jderose/BCC/Chinchilla/Herd/Chinchilla-4/v1.9.2/addgalspostprocess/truth/truth/Chinchilla-4_lensed.4.fits"

In [ ]:
simhdu = fits.open(simname)
truthhdu = fits.open(truthname)
sim_table = simhdu[1].data
truth_table = truthhdu[1].data
#simhdu.info()
#print(sim_table.columns);
#truthhdu.info()
#print(truth_table.columns)

In [ ]:
cat_file_name = os.path.join('/nfs/slac/des/fs1/g/sims/jderose/BCC/Chinchilla/Herd/Chinchilla-4/v1.9.2/addgalspostprocess/truth/truth/','Chinchilla-4_lensed.4.fits')
cat = galsim.Catalog(cat_file_name)

In [ ]:
noisy_file_name = os.path.join('/afs/slac.stanford.edu/u/ki/gwisk/CRNProject','noisy.fits')
noisefree_file_name = os.path.join('/afs/slac.stanford.edu/u/ki/gwisk/CRNProject','noisefree.fits')

rband = truth_table['LMAG'][:,1] #grizy
_ra = truth_table['RA']
_dec = truth_table['DEC']
_hlr = truth_table['SIZE']
te1 = truth_table['EPSILON'][:,0]
te2 = truth_table['EPSILON'][:,1]

# Timing Tests

In [ ]:
gaussiannoise = galsim.GaussianNoise(sigma=0.005)
x, y = np.mgrid[-24.5:25.5:1, -24.5:25.5:1]
pos = np.zeros(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y
#_nimages, _nfimages, _nmoments, _nfmoments, shearmatrices, noisematrices, _gfluxes, _hlr, failures =([] for i in range(9))

def HSMfunc(integer):
    failures = []
    _nmoments = []
    _gfluxes = []
    for i in range(integer):
        image = galsim.ImageF(50, 50)
        # catalogs
        hlradius = _hlr[i]   
        elip1 = te1[i]
        elip2 = te2[i]
        lmag = rband[i]
        gflux = 10**((lmag-22.5)/(-2.5))
        _gfluxes.append(gflux)
        #make galaxy
        gal = galsim.Gaussian(flux=gflux, half_light_radius=hlradius)
        # shearing
        #shearing1 = galsim.Shear(e1=elip1, e2=elip2)
        gal = gal.shear(galsim.Shear(e1=elip1, e2=elip2))
        # create image, convolve
        
        final = galsim.Convolve([gal])
        image = final.drawImage(scale=0.2)
        image.addNoise(gaussiannoise)
        
    
        #HSM
        nmoment = image.FindAdaptiveMom(strict = False, precision = 1e-3) #,guess_sig=hlradius*5)
        _nmoments.append(nmoment)
        if nmoment.moments_n_iter == 0:
            failures.append(i)
    return failures, _nmoments, _gfluxes

def cheatfunc(integer):
    shearmatrices = []
    noisematrices = []
    for i in range(integer):
        noiseimage = galsim.ImageF(50, 50)
        # catalogs
        ra = cat.get(i, 'RA')
        dec = cat.get(i, 'DEC')
        hlradius = cat.get(i, 'SIZE')
        
        # more catalogs
        elip1 = te1[i]
        elip2 = te2[i]
        lmag = rband[i]
        gflux = 10**((lmag-22.5)/(-2.5))

        shearing2 = galsim.Shear(e1=-1*elip1, e2=elip2)
        matrix = (shearing2.getMatrix())*(hlradius**2)*25
        shearmatrices.append(matrix)

        noiseimage.addNoise(gaussiannoise)
        noisematrices.append(noiseimage.array)
        
    return shearmatrices, noisematrices

In [ ]:
%%timeit
shearmatrices, noisematrices = cheatfunc(3000)
cheatflux = np.zeros(3000)
# look in the book Pat sent
for j in range(foobar):
    rv = multivariate_normal([0, 0], shearmatrices[j])
    cheatflux[j] = np.sum(np.multiply(rv.pdf(pos), noisematrices[j])/np.sum(rv.pdf(pos)))
#2.45 s ± 307 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%timeit
failures, _nmoments, _gfluxes = HSMfunc(3000)
nmoments = [i for j, i in enumerate(_nmoments) if j not in failures]
gfluxes = [i for j, i in enumerate(_gfluxes) if j not in failures]
HSMflux = np.zeros(len(nmoments))
for k in range(len(nmoments)):
    HSMflux[k] = nmoments[k].moments_amp - gfluxes[k]
    #10.8 s ± 1.77 s per loop (mean ± std. dev. of 7 runs, 1 loop each) for 3000 runs, default precision 

# Tests Below

In [ ]:
noisy_image_file = get_pkg_data_filename('/afs/slac.stanford.edu/u/ki/gwisk/CRNProject/noisy.fits')

In [ ]:
HSMflux = np.zeros(len(nimages))
#HSMflux1 = np.zeros(len(nimages))
#x, y = np.mgrid[-5:5:.2, -5:5:.2]
cheatflux = np.zeros(len(nimages))
sigmas = []

x, y = np.mgrid[-24.5:25.5:1, -24.5:25.5:1]
pos = np.zeros(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y
for k in range(len(nimages)):
    HSMflux[k] = (nmoments[k].moments_amp) #-  nfmoments[k].moments_amp)# -  gfluxes[k]))
   # HSMflux1[k] = nfmoments[k].moments_amp

for j in range(len(nimages)):
    rv = multivariate_normal([0, 0], shearmatrices[j])
    foo = np.sum(np.multiply (rv.pdf(pos), noisematrices[j] + nfimages[j].array)/np.sum(rv.pdf(pos)))
    foo = np.sum(np.multiply (rv.pdf(pos), nfimages[j].array))
    cheatflux[j] = foo

plt.figure()
plt.scatter(cheatflux, gfluxes)


In [ ]:
plt.figure(1)
fluxes = []
f, ax = plt.subplots(3, 3, figsize=(11, 9))
for i in range(9):
    image_data = fits.getdata(noisy_image_file, ext = i)
    im = ax[i//3, i%3].imshow(image_data, origin="lower")
    f.colorbar(im, ax=ax[i//3, i%3])
plt.tight_layout(1)

In [ ]:
plt.figure(2)
g, ax = plt.subplots(3, 3, figsize=(11, 9))
#x, y = np.mgrid[-.24:.24:.01, -.24:.24:.01]
x, y = np.mgrid[-.5:.5:.02, -.5:.5:.02]

pos = np.zeros(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y
#np.multiply(rv.pdf(pos),
for j in range(0, 9):
    rv = multivariate_normal([0, 0], shearmatrices[j])
    im = ax[j//3, j%3].imshow(np.multiply(rv.pdf(pos), noisematrices[j] + nfimages[j].array), origin = "lower")
    #im = ax[j//3, j%3].imshow(noisematrices[j], origin = "lower")
    g.colorbar(im, ax=ax[j//3, j%3])
plt.tight_layout(2)

In [ ]:
# half light radius squared/flux
_nimages, _nfimages, _nmoments, _nfmoments, _shearmatrices, _noisematrices, _gfluxes, _hlr, failures =([] for i in range(9))
def imagefunc(integer):
    for i in range(integer):
        # catalogs
        ra = cat.get(i, 'RA')
        dec = cat.get(i, 'DEC')
        hlradius = cat.get(i, 'SIZE')
        _hlr.append(hlradius)
        
        # more catalogs
        elip1 = te1[i]
        elip2 = te2[i]
        lmag = rband[i]
        gflux = 10**((lmag-22.5)/(-2.5))
        _gfluxes.append(gflux)
        #make galaxy
        gal = galsim.Gaussian(flux=gflux, half_light_radius=hlradius)
        # shearing
        shearing1 = galsim.Shear(e1=elip1, e2=elip2)
        shearing2 = galsim.Shear(e1=-1*elip1, e2=elip2)
        gal = gal.shear(shearing1)
        matrix = (shearing2.getMatrix())*(hlradius**2)*25
        _shearmatrices.append(matrix)

        # create image, convolve

        nfimage = galsim.ImageF(50, 50)
        nimage = galsim.ImageF(50, 50)
        noisefreefinal = galsim.Convolve([gal])
        noisefinal = galsim.Convolve([gal])
        noisefreefinal.drawImage(nfimage, scale=0.2)
        noisefinal.drawImage(nimage, scale=0.2)
        #noisefree
        nfmoment = galsim.hsm.FindAdaptiveMom(nfimage, strict = False, guess_sig = hlradius*5)
        _nfmoments.append(nfmoment)
        _nfimages.append(nfimage)
        #noisy
        nimage.addNoise(galsim.GaussianNoise(sigma=0.005))
        _nimages.append(nimage)
        nmoment = galsim.hsm.FindAdaptiveMom(nimage, strict = False,  guess_sig=hlradius*5)
        _nmoments.append(nmoment)

        if nmoment.moments_n_iter == 0:
            failures.append(i)
        noiseimage = galsim.ImageF(50, 50)
        noise = galsim.GaussianNoise(sigma=0.005)
        noiseimage.addNoise(noise)
        _noisematrices.append(noiseimage.array)
    #print(np.mean(hlr))
    galsim.fits.writeMulti(_nimages, noisy_file_name)
    return _nimages, _nfimages, _nmoments, _nfmoments, _shearmatrices, _noisematrices, _gfluxes, _hlr, failures


imagefunc(500)
nimages = [i for j, i in enumerate(_nimages) if j not in failures]
nmoments = [i for j, i in enumerate(_nmoments) if j not in failures]
nfmoments = [i for j, i in enumerate(_nfmoments) if j not in failures]
shearmatrices = [i for j, i in enumerate(_shearmatrices) if j not in failures]
noisematrices = [i for j, i in enumerate(_noisematrices) if j not in failures]
gfluxes = [i for j, i in enumerate(_gfluxes) if j not in failures]
hlr = [i for j, i in enumerate(_hlr) if j not in failures]
nfimages = [i for j, i in enumerate(_nfimages) if j not in failures]